# VVP Validation & Verification Protocol Tutorial

This tutorial demonstrates SIM-ONE's VVP protocol for validating logical rule structures before they're used in reasoning operations.

## What You'll Learn
- Validate logical rule syntax and structure
- Check for rule conflicts and inconsistencies
- Verify rule completeness and coverage
- Ensure rules are properly formed for reasoning engines

## Why Validate Rules?

Before performing logical reasoning (especially deductive reasoning), it's critical to ensure that:

1. **Syntax is correct** - Rules follow proper logical format
2. **No conflicts exist** - Rules don't contradict each other
3. **Structure is complete** - All necessary components are present
4. **Semantics are valid** - Rules make logical sense

## Use Cases

- **Rule-based systems** - Validate expert system rules before deployment
- **Deductive reasoning** - Ensure reasoning rules are properly formed
- **Policy enforcement** - Verify compliance rules are consistent
- **Logic programming** - Check logical programs for errors

In [ ]:
# Standard library imports
import sys
import json
from pathlib import Path
from typing import Dict, Any, Optional, List
import pandas as pd

# Add SIM-ONE to Python path
SIMONE_ROOT = Path("../code").resolve()
sys.path.insert(0, str(SIMONE_ROOT))

# Import VVP protocol directly
from mcp_server.protocols.vvp.vvp import VVP

print("✓ VVP (Validation & Verification Protocol) initialized")

In [ ]:
# Initialize VVP protocol
vvp = VVP()

print("✓ VVP Protocol initialized")
print("  Supports: Rule structure validation, conflict detection, completeness checking")

## Rule Format Reference

VVP validates rules in the following format:

```python
{
    "rules": [
        # Each rule: [[list of premises], conclusion]
        [["premise1", "premise2"], "conclusion"],
        [["premise3"], "conclusion2"]
    ]
}
```

### Valid Rule Examples

**Simple Rule**:
```python
[["All humans are mortal", "Socrates is a human"], "Socrates is mortal"]
```

**Multi-Premise Rule**:
```python
[["Sky is blue", "Grass is green", "Sun is bright"], "It is daytime"]
```

**Single-Premise Rule**:
```python
[["Temperature above 100°C"], "Water boils"]
```

### Invalid Rule Examples

**Missing premises** (empty list):
```python
[[], "Some conclusion"]  # ❌ INVALID
```

**Missing conclusion**:
```python
[["Some premise"], ""]  # ❌ INVALID
```

**Wrong structure** (not a list):
```python
["premise", "conclusion"]  # ❌ INVALID (should be [[premises], conclusion])
```

## Example 1: Validating Correct Rule Structures

Let's validate a properly formatted rule set.

In [ ]:
# Define valid rule structure
rules_data = {
    "rules": [
        [["All mammals are warm-blooded", "Dogs are mammals"], "Dogs are warm-blooded"],
        [["All birds have feathers", "Penguins are birds"], "Penguins have feathers"],
        [["If it rains, ground is wet", "It is raining"], "Ground is wet"]
    ]
}

# Validate rules using VVP
result = vvp.execute(rules_data)

print("VVP Validation Result:")
print("=" * 80)
print(json.dumps(result, indent=2))

print(f"\n✓ Validation Status: {result.get('status', 'Unknown')}")
print(f"  Total Rules: {result.get('total_rules', 0)}")
print(f"  Valid Rules: {result.get('valid_rules', 0)}")

if result.get("validation_details"):
    print(f"\n📋 Validation Details:")
    for i, detail in enumerate(result["validation_details"], 1):
        print(f"  Rule {i}: {detail.get('status', 'Unknown')}")
        if detail.get("message"):
            print(f"    Message: {detail['message']}")

## Example 2: Detecting Invalid Rule Structures

Now let's see how VVP catches malformed rules.

In [ ]:
# Define rules with various structural issues
invalid_rules = {
    "rules": [
        # Valid rule
        [["All cats are animals"], "Fluffy is an animal"],

        # INVALID: Empty premises list
        [[], "Something is true"],

        # INVALID: Missing conclusion (empty string)
        [["Some premise"], ""],

        # INVALID: Wrong structure (not nested list)
        ["premise", "conclusion"],

        # Valid rule
        [["X implies Y", "X is true"], "Y is true"]
    ]
}

# Validate rules
result = vvp.execute(invalid_rules)

print("VVP Validation Result (Invalid Rules):")
print("=" * 80)

print(f"Status: {result.get('status', 'Unknown')}")
print(f"Total Rules: {result.get('total_rules', 0)}")
print(f"Valid Rules: {result.get('valid_rules', 0)}")
print(f"Invalid Rules: {result.get('invalid_rules', 0)}")

if result.get("validation_details"):
    print(f"\n📋 Detailed Validation:")
    for i, detail in enumerate(result["validation_details"], 1):
        status_symbol = "✓" if detail.get("status") == "valid" else "✗"
        print(f"  {status_symbol} Rule {i}: {detail.get('status', 'Unknown')}")
        if detail.get("error"):
            print(f"    Error: {detail['error']}")
        if detail.get("message"):
            print(f"    Message: {detail['message']}")

if result.get("errors"):
    print(f"\n❌ Errors Summary:")
    for error in result["errors"]:
        print(f"  - {error}")

## Example 3: Detecting Rule Conflicts

VVP can identify when rules contradict each other.

In [ ]:
# Define rules that conflict with each other
conflicting_rules = {
    "rules": [
        [["All birds can fly"], "Penguins can fly"],
        [["Penguins are birds"], "Penguins exist"],
        [["Penguins cannot fly"], "Penguins are flightless"],  # Conflicts with rule 1
        [["If X can fly, X has wings"], "Penguins have wings"]
    ]
}

# Validate rules
result = vvp.execute(conflicting_rules)

print("VVP Conflict Detection:")
print("=" * 80)

print(f"Status: {result.get('status', 'Unknown')}")
print(f"Total Rules: {result.get('total_rules', 0)}")

if result.get("conflicts"):
    print(f"\n⚠ Conflicts Detected ({len(result['conflicts'])}):")
    for i, conflict in enumerate(result["conflicts"], 1):
        print(f"\n  Conflict {i}:")
        print(f"    Rule A: {conflict.get('rule_a')}")
        print(f"    Rule B: {conflict.get('rule_b')}")
        print(f"    Issue: {conflict.get('description')}")
else:
    print("\n✓ No conflicts detected")

if result.get("warnings"):
    print(f"\n💡 Warnings:")
    for warning in result["warnings"]:
        print(f"  - {warning}")

## Example 4: Checking Rule Set Completeness

Ensure rule sets cover all necessary cases without gaps.

In [ ]:
# Define rule set for checking completeness
rules_with_gaps = {
    "rules": [
        [["Temperature > 0°C", "Pressure = 1 atm"], "Water is liquid"],
        [["Temperature > 100°C", "Pressure = 1 atm"], "Water is gas"],
        # Missing: What about temperature < 0°C? (water is solid/ice)
    ],
    "context": {
        "domain": "Water phase transitions",
        "expected_coverage": ["solid", "liquid", "gas"]
    }
}

# Validate rules
result = vvp.execute(rules_with_gaps)

print("VVP Completeness Check:")
print("=" * 80)

print(f"Status: {result.get('status', 'Unknown')}")

if result.get("completeness"):
    comp = result["completeness"]
    print(f"\nCompleteness Analysis:")
    print(f"  Coverage: {comp.get('coverage_percentage', 0):.1f}%")
    print(f"  Covered Cases: {comp.get('covered_cases', [])}")
    print(f"  Missing Cases: {comp.get('missing_cases', [])}")

if result.get("suggestions"):
    print(f"\n💡 Suggestions for Improvement:")
    for suggestion in result["suggestions"]:
        print(f"  - {suggestion}")

## Example 5: Validating Rules from Files

VVP can validate rules stored in JSON files.

In [ ]:
# Create sample rules file
sample_rules_file = Path("sample_rules.json")

sample_rules = {
    "rules": [
        [["All employees must clock in", "John is an employee"], "John must clock in"],
        [["Meeting starts at 9 AM", "It is 9 AM"], "Meeting is starting"],
        [["If urgent, escalate immediately"], "Handle urgent matters promptly"]
    ]
}

# Write to file
sample_rules_file.write_text(json.dumps(sample_rules, indent=2))

# Validate rules from file
rules_from_file = json.loads(sample_rules_file.read_text())
result = vvp.execute(rules_from_file)

print(f"VVP Validation from File: {sample_rules_file}")
print("=" * 80)

print(f"Status: {result.get('status', 'Unknown')}")
print(f"Total Rules: {result.get('total_rules', 0)}")
print(f"Valid Rules: {result.get('valid_rules', 0)}")

# Create validation summary table
if result.get("validation_details"):
    summary_data = []
    for i, detail in enumerate(result["validation_details"], 1):
        rule_text = f"Rule {i}"
        if detail.get("rule"):
            premises = detail["rule"].get("premises", [])
            conclusion = detail["rule"].get("conclusion", "")
            rule_text = f"{' & '.join(premises[:2])} → {conclusion[:30]}"

        summary_data.append({
            "Rule": f"Rule {i}",
            "Status": detail.get("status", "Unknown"),
            "Structure": "✓" if detail.get("structure_valid") else "✗",
            "Semantics": "✓" if detail.get("semantics_valid") else "✗"
        })

    df = pd.DataFrame(summary_data)
    print("\n" + df.to_string(index=False))

# Cleanup
sample_rules_file.unlink()
print(f"\n✓ Validation complete, file cleaned up")

## Example 6: VVP + REP Integration

Demonstrate how VVP validates rules before they're used in REP deductive reasoning.

In [ ]:
# Import REP for integration
from mcp_server.protocols.rep.rep import AdvancedREP

rep = AdvancedREP()

# Define rules to validate and use
rules_data = {
    "rules": [
        [["All machines need maintenance", "Robot A is a machine"],
         "Robot A needs maintenance"],
        [["If maintenance overdue, schedule service", "Robot A maintenance overdue"],
         "Schedule service for Robot A"]
    ]
}

print("Step 1: Validate rules with VVP")
print("=" * 80)

# Validate first
vvp_result = vvp.execute(rules_data)
print(f"VVP Status: {vvp_result.get('status', 'Unknown')}")
print(f"Valid Rules: {vvp_result.get('valid_rules', 0)}/{vvp_result.get('total_rules', 0)}")

if vvp_result.get("status") == "valid":
    print("\n✓ Rules validated successfully")

    print("\nStep 2: Use validated rules in REP deductive reasoning")
    print("=" * 80)

    # Extract validated rules for REP
    facts = [
        "All machines need maintenance",
        "Robot A is a machine",
        "If maintenance overdue, schedule service",
        "Robot A maintenance overdue"
    ]

    rules = [
        [["All machines need maintenance", "Robot A is a machine"],
         "Robot A needs maintenance"],
        [["If maintenance overdue, schedule service", "Robot A maintenance overdue"],
         "Schedule service for Robot A"]
    ]

    # Perform deductive reasoning with validated rules
    rep_result = rep.perform_reasoning(
        reasoning_type="deductive",
        facts=facts,
        rules=rules,
        context="Maintenance scheduling system"
    )

    print(f"REP Status: Success")
    print(f"Conclusions: {rep_result.get('conclusions', [])}")

    print("\n✓ VVP + REP Integration Complete")
    print("  Rules were validated by VVP before being used in REP reasoning")
else:
    print("\n✗ Rules failed validation - cannot proceed with reasoning")
    print(f"  Errors: {vvp_result.get('errors', [])}")

## Summary

This tutorial demonstrated SIM-ONE's VVP protocol for rule validation:

### Key Capabilities

1. **Structure Validation** - Ensures rules follow proper format
2. **Conflict Detection** - Identifies contradicting rules
3. **Completeness Checking** - Verifies rule coverage
4. **Pre-Reasoning Validation** - Validates rules before REP usage

### When to Use VVP

- **Before deductive reasoning** - Validate rules before applying them
- **Rule base development** - Check rules during expert system creation
- **Policy verification** - Ensure business rules are consistent
- **Logic debugging** - Find errors in logical rule sets

### VVP Validation Checks

| Check Type | Purpose | Detects |
|------------|---------|---------||
| **Structure** | Syntax correctness | Empty premises, missing conclusions, wrong format |
| **Conflicts** | Logical consistency | Contradicting rules, circular dependencies |
| **Completeness** | Coverage | Missing cases, gaps in rule coverage |
| **Semantics** | Logical validity | Nonsensical rules, impossible conditions |

### API Reference

```python
from mcp_server.protocols.vvp.vvp import VVP

vvp = VVP()

# Validate rules
result = vvp.execute({
    "rules": [
        [[premises_list], conclusion],
        ...
    ],
    "context": {  # optional
        "domain": "...",
        "expected_coverage": [...]
    }
})
```

### Output Structure

```json
{
  "status": "valid" | "invalid" | "warning",
  "total_rules": 5,
  "valid_rules": 4,
  "invalid_rules": 1,
  "validation_details": [
    {
      "rule_index": 0,
      "status": "valid",
      "structure_valid": true,
      "semantics_valid": true,
      "message": "..."
    }
  ],
  "conflicts": [...],
  "errors": [...],
  "warnings": [...],
  "suggestions": [...]
}
```

### Integration Pattern

```
1. Define Rules
   ↓
2. VVP Validation ← You are here
   ↓
3. If Valid → REP Deductive Reasoning
   ↓
4. Logical Conclusions
```

### Best Practices

1. **Always validate before reasoning** - Catch errors early
2. **Check for conflicts** - Prevent contradictory conclusions
3. **Verify completeness** - Ensure all cases are covered
4. **Provide context** - Help VVP understand domain-specific rules
5. **Review suggestions** - Improve rule quality based on VVP feedback